Collect the following data 
1. equity (snp500 + world)
2. FI (T-bill+corpbonds index)
3. commodity (commodity index)
4. gold (london gold index)
5. long-vol (eurohedge log vol)   

1. '^SP500TR' (total return inc. div, oldest up to 1988) + 'EEM' (oldest up to 14 apr 2003)
2. 'IEF' (oldest up to 2002) + 'AGG' (up to 29 sep 2003 !dividend distributed)
3. 'BCOM' (up to 1991)
4. 'LBMA Gold Price' excel (1988) -  https://www.gold.org/goldhub/data/gold-prices
5. 'Euro hedge long-vol' excel (2005, monthly pricing!) - https://www.eurekahedge.com/Indices/IndexView/Eurekahedge/640/CBOE-Eurekahedge-Long-Volatility-Hedge-Fund-Index

### Date Range
2003-9-29
2021-9-31

In [3]:
import pandas_datareader.data as web
from pandas_datareader.yahoo.headers import DEFAULT_HEADERS
import datetime
import requests_cache

import numpy as np, pandas as pd

# Data from Yahoo finance

In [33]:
# list of tickers required from yahoo finance
tickers = ['^SP500TR','EEM','IEF' , 'AGG','^BCOM']

In [45]:
# pd datareader with yahoo finance, caching query to sql lite file
expire_after = datetime.timedelta(days=3)
session = requests_cache.CachedSession(cache_name='cache', backend='sqlite', expire_after=expire_after)
session.headers = DEFAULT_HEADERS

start = datetime.datetime(2003, 9, 29)

end = datetime.datetime(2021, 10, 1)
data = web.DataReader(tickers, 'yahoo', start, end, session=session)

In [46]:
data.head()

Attributes    Adj Close                                               \
Symbols        ^SP500TR        EEM        IEF        AGG       ^BCOM   
Date                                                                   
2003-09-29  1462.199951  10.932035  52.032452  57.497845  120.430000   
2003-09-30  1446.770020  10.897563  52.536015  57.796089  120.900002   
2003-10-01  1479.229980  11.183772  52.769131  57.767998  121.139999   
2003-10-02  1482.359985  11.344110  52.579453  57.677971  121.930000   
2003-10-03  1496.319946  11.456348  51.851284  57.261505  121.940002   

Attributes        Close                                                ...  \
Symbols        ^SP500TR        EEM        IEF         AGG       ^BCOM  ...   
Date                                                                   ...   
2003-09-29  1462.199951  15.151111  85.760002  102.169998  120.430000  ...   
2003-09-30  1446.770020  15.103333  86.589996  102.699997  120.900002  ...   
2003-10-01  1479.229980  15.500000  86.239998  102.650002  121.139999  ...   
2003-10-02  1482.359985  15.722222  85.930000  102.489998  121.930000  ...   
2003-10-03  1496.319946  15.877778  84.739998  101.750000  121.940002  ...   

Attributes         Open                                                \
Symbols        ^SP500TR        EEM        IEF         AGG       ^BCOM   
Date                                                                    
2003-09-29  1462.199951  15.068889  85.900002  102.290001  119.620003   
2003-09-30  1446.770020  15.245556  85.940002  102.300003  120.720001   
2003-10-01  1479.229980  15.312222  86.269997  102.639999  120.849998   
2003-10-02  1482.359985  15.645556  85.730003  102.199997  122.000000   
2003-10-03  1496.319946  15.944444  85.169998  102.050003  121.690002   

Attributes   Volume                                      
Symbols    ^SP500TR        EEM       IEF      AGG ^BCOM  
Date                                                     
2003-09-29      0.0  1071000.0   97500.0  13600.0   0.0  
2003-09-30      0.0  2218500.0  450400.0  62600.0   0.0  
2003-10-01      0.0  1123200.0  102100.0  66300.0   0.0  
2003-10-02      0.0  1017900.0  163500.0  68900.0   0.0  
2003-10-03      0.0  2016900.0  319500.0  64500.0   0.0  

[5 rows x 30 columns]

In [47]:
# use adjusted close as daily px
data = data['Adj Close']

# get log pct change as returns each day
data = data.pct_change().apply(lambda x: np.log(1+x))

In [48]:
data.head()

Symbols,^SP500TR,EEM,IEF,AGG,^BCOM
Date,,,,,
2003-09-29,NaN,NaN,NaN,NaN,NaN
2003-09-30,-0.010609,-0.003158,0.009631,0.005174,0.003895
2003-10-01,0.022188,0.025925,0.004427,-0.000486,0.001983
2003-10-02,0.002114,0.014235,-0.003601,-0.001560,0.006500
2003-10-03,0.009373,0.009845,-0.013946,-0.007247,0.000082


In [82]:
data.isnull().sum()

Symbols
^SP500TR    1
EEM         1
IEF         1
AGG         1
^BCOM       1
dtype: int64

# Data from Websites

In [83]:
long_vol = pd.read_excel('./data/long-vol.xlsx')  
long_vol.columns = long_vol.iloc[2]
long_vol = long_vol[3:].set_index('ReturnDate')['PercentReturn']

In [85]:
long_vol.min()

-4.98858528860432

In [86]:
np.log(-4)

<ipython-input-86-bb463563359a>:1: RuntimeWarning: invalid value encountered in log
  np.log(-4)


nan

In [97]:
gold = pd.read_excel('./data/gold.xlsx',sheet_name='Daily_Indexed')
gold = gold[['Name', 'US dollar']].set_index('Name')
gold = gold.pct_change().apply(lambda x:np.log(1+x))